In [8]:
from folium.plugins import MarkerCluster
import pandas as pd
import folium

# 데이터 로딩
df = pd.read_csv("../data/Data_saramin_with_coordinates.csv", encoding='utf-8')
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df_valid = df.dropna(subset=['위도', '경도'])

# 지도 초기화
map_center = df_valid[['위도', '경도']].mean().tolist()
m = folium.Map(location=map_center, zoom_start=11)

# 클러스터 생성
marker_cluster = MarkerCluster().add_to(m)

# 마커 추가 (클러스터에)
for idx, row in df_valid.iterrows():
    lat, lon = row['위도'], row['경도']
    address = row.get('주소', '주소 없음')

    folium.Marker(
        [lat, lon]
        #popup=f"Index: {idx}<br>주소: {address}",
        #tooltip=address
    ).add_to(marker_cluster)

# 저장
m.save("company_address_map.html")
print("✅ 클러스터 지도 저장 완료: company_address_map.html")


✅ 클러스터 지도 저장 완료: company_address_map.html


In [5]:
from geopy.distance import geodesic
import pandas as pd
import folium

# --------------------------
# 데이터 불러오기 및 전처리
# --------------------------
# 1. 상위 30개 혼잡 역 (위경도 포함)
toprank_df = pd.read_csv("../data/subway_complexity_toprank_대표호선.csv", encoding='utf-8')
coord_df = pd.read_csv("../data/지하철_1_8호선 역주소(위경도)_호선통합.csv", encoding='euc-kr')
addr_df = pd.read_csv("../data/지하철_역주소 및 전화번호_부기역명제거.csv", encoding='euc-kr')
addr_df = addr_df[['호선', '역명', '도로명주소']]
merged_df = pd.merge(coord_df, addr_df, on='역명', how='left')
filtered_df = merged_df[merged_df['도로명주소'].str.startswith('서울특별시', na=False)]
toprank_df['total_commute'] = toprank_df['morning'] + toprank_df['evening']
grouped_toprank = toprank_df.groupby('역명', as_index=False).agg({
    'morning': 'sum', 'evening': 'sum', 'total_commute': 'sum'
})
top30_df = grouped_toprank.sort_values(by='total_commute', ascending=False).head(30)
subway_df = pd.merge(top30_df, filtered_df, on='역명', how='inner')

# 2. 기업 위치 데이터
company_df = pd.read_csv("../data/Data_saramin_with_coordinates.csv", encoding='utf-8')
company_df['위도'] = pd.to_numeric(company_df['위도'], errors='coerce')
company_df['경도'] = pd.to_numeric(company_df['경도'], errors='coerce')
company_df = company_df.dropna(subset=['위도', '경도'])

# --------------------------
# 지도 생성
# --------------------------
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# --------------------------
# 각 지하철 역에 대해 기업 수 계산 + 마커 표시
# --------------------------
for _, row in subway_df.iterrows():
    station_coord = (row['위도'], row['경도'])
    station_name = row['역명']
    morning = row['morning']
    evening = row['evening']
    total_commute = row['total_commute']

    # ✅ 600m 이내 기업 수 계산
    radius_m = 600
    nearby_count = company_df.apply(
        lambda x: geodesic(station_coord, (x['위도'], x['경도'])).meters <= radius_m,
        axis=1
    ).sum()

    # 마커 팝업 텍스트
    popup_text = f"""
    <b>{station_name}</b><br>
    출근: {morning:,}명<br>
    퇴근: {evening:,}명<br>
    합계: {total_commute:,}명<br>
    <b>기업 수 (600m 이내):</b> {nearby_count}개
    """

    folium.Circle(
        location=station_coord,
        radius=radius_m,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.1,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# --------------------------
# 저장
# --------------------------
m.save("서울_지하철역_혼잡도_기업수.html")
print("✅ 저장 완료: 서울_지하철역_혼잡도_기업수.html")


✅ 저장 완료: 서울_지하철역_혼잡도_기업수.html
